In [1]:
import time
import math
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from sklearn.metrics import confusion_matrix
from datetime import timedelta
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

In [ ]:
labels=[]
images=[]
for i in range(10000):
    file_name = "../data_set/test_images1/anim{}.png".format(i)
    img = cv2.imread(file_name)
    images.append(img.tolist())
    labels.append(1)
for i in range(10000):
    file_name = "../data_set/test_images2/anim{}.png".format(i)
    img = cv2.imread(file_name)
    images.append(img.tolist())
    labels.append(2)
for i in range(5000):
    file_name = "../data_set/test_images3/anim{}.png".format(i)
    img = cv2.imread(file_name)
    images.append(img.tolist())
    labels.append(3)

In [11]:
%matplotlib inline
images = np.array(images)

In [12]:
images.shape

(10000, 307, 400, 3)